In [ ]:
%load hicrep.py

### Set global parameters

In [1]:
# Set some global parameters
max_dist = 5000000
h_max = 10
resolution = 50000

whitelist = ["chr3", "chr9", "chr14"] # can pick a random sampling of chromosomes for speed

### Read mcools

In [ ]:
file_path = "" # path to mcools
mcool_files = [""] # list of mcool files 
multi_coolers = [read_cooler(os.path.join(file_path, mcool_file), resolution) for mcool_file in mcool_files] # read mcool files at resolution
h_train_idxs = [0, 1]

### Find optimal h value

In [ ]:
optimal_h = h_train(multi_coolers[h_train_idxs[0]], multi_coolers[h_train_idxs[1]], max_dist, h_max, whitelist)

### Compute scc values

In [ ]:
# compute downsampling value
cooler_sums = [cooler_file.info["sum"] for cooler_file in multi_coolers[:-1]]
downsampling_value = min(cooler_sums)

k = len(multi_coolers)
scc_mat = np.zeros((k, k))
for i, j in tqdm(product(range(k), range(k))):
    if i == j:
        scc_mat[i, j] = 1.0
    else:
        scc = genome_scc(multi_coolers[i], multi_coolers[j], max_dist, optimal_h, downsampling_value, whitelist)
        scc_mat[i, j] = scc
        scc_mat[j, i] = scc

### Plot correlation matrices

In [ ]:
plot_dir = "" # output file directory
sns.clustermap(scc_df, cmap= sns.color_palette("RdBu_r", as_cmap=True), annot=True)
plt.savefig(plot_dir, format='svg')